# CTD LISST Comparison

Intro: A Sequoia Scientific LISST-200X is mounted on the CTD rosette during Pioneer MAB overturning cruises for comparison with the moored PRTSZ instruments on the OOI Coastal Surface Moorings. Questionable data from the moorings shortly after deployment in Spring 2025, prompted comparison and identification of a similar issue in the CTD cast data. The CTD-mounted LISST data quality was improved with thorough cleaning and a new clean water background which resulted in lower background counts in most of the 36 detector bins. 

Issue: Whether the updated clean water background can be reasonably used in processing of the first 20 casts of the Pioneer-21 Leg 1 cruise (AR87a) has not yet been confirmed, despite processing showing reasonable beam attenuation and optical transmission measurements.

Purpose: Compare LISST data collected during CTD casts with beam attenuation from the CTD transmissometer for data quality validation. Processed LISST data using the first clean water background of the cruise (high background counts, negative beam attenuation) and the second clean water background taken mid-cruise (low background counts, positive beam attenuation) will be compared.

#### Import modules used in analysis

In [ ]:
from os import path

import pandas as pd
from ioos_qc.qartod import gross_range_test # guessing at this

#### Load data into workspace
Load processed CSV data files from the LISST-200X mounted on the CTD rosette for Cast 005 from AR87a. 

In [ ]:
# Load LISST CSV column names from json containing column headers
csvhdr = pd.read_json("./inst_headers/lisst_hdr.json")

In [ ]:
# Define paths to data
cruise_data_drive = "D:/"
lisst_onboard_path = "LISST_PRTSZ/Pioneer21_AR87_LISST_data/AR87a_CAST005_SO/*.csv"
lisst_reproc_path = "LISST_PRTSZ/Pioneer21_AR87_LISST_data/Processed_Data_for_QC/20250404_Reprocessed/L0900037*.csv"
ctd_inst_path = "general*/AR87a_CTD_Casts/ctd/process/*_005.asc"

In [ ]:
# Load LISST data processed with onboard clean water background
lisst_onboard = pd.read_csv(path.join(cruise_data_drive, lisst_onboard_path), header=)
lisst_reproc = pd.read_csv(path.join(cruise_data_drive, lisst_reproc_path))
ctd_inst = pd.read_csv(ctd_inst_path, header=0)

In [ ]:
# Load LISST data processed with updated clean water background

In [ ]:
# Load transmissometer data

#### Plot Transmissometer and LISST data

2 plots:<ul>
    <li> Transmissometer and LISST data with 2 different clean water backgrounds
    <li> Difference between each pair: <ul>
        <li> Transmissometer and original LISST
        <li> Transmissometer and updated LISST
        <li> Original LISST and updated LISST

#### Evaluate quality tests on LISST data
The recommended QC procedure for optical transmission from Sequoia Scientific is encapsulated by running the IOOS QARTOD gross range test with both "fail" and "of interest/suspect" spans coming from the instrument manual. Other parameters are QC'ed with a gross range test which checks for values that fall within the vendor-defined instrument limits.

#### Plot profiles of total volume concentration
Using the data flags derived from optical transmission, we will plot total volume concentration, aka the sum of the individual volume concentration bins, for data that passes the quality tests. Suspect points will be plotted in yellow (for "caution").